In [ ]:
!nvidia-smi

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
import prompt
import dataset

import os
import preprocess
import numpy as np

import pydicom
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig
from skimage.filters import threshold_otsu, gaussian, median, unsharp_mask
from skimage.measure import label, regionprops

In [ ]:
import torch
torch.manual_seed(1234)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
CACHE_DIR = "/root/letractien/Mammo-VLM/.cache"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-VL-Chat", trust_remote_code=True, cache_dir=CACHE_DIR)
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-VL-Chat", device_map=device, trust_remote_code=True, cache_dir=CACHE_DIR).eval()
model.generation_config = GenerationConfig.from_pretrained("Qwen/Qwen-VL-Chat", trust_remote_code=True, cache_dir=CACHE_DIR)

In [ ]:
image_annotation_tuples = dataset.load_image_annotation_tuples()
unique_tuples = list({img_path: (img_path, ann) for img_path, ann in image_annotation_tuples}.values())

save_dir = "out/detect_qwen_with_preprocess_ipynb"
os.makedirs(save_dir, exist_ok=True)
log_path = os.path.join(save_dir, "log.txt")

# Run pipeline

In [ ]:
for idx, (img_path, annotation) in enumerate(unique_tuples):

    folder = annotation['study_id']
    os.makedirs(os.path.join(save_dir, folder), exist_ok=True)

    basename = annotation['image_id']
    img_png_path = os.path.join(save_dir, folder, f"{basename}.png")

    ds = pydicom.dcmread(img_path)
    # plt.imsave(img_png_path, ds.pixel_array, cmap="gray")

    try: img_arr = ds.pixel_array.astype(np.float32)
    except Exception as e: continue 

    # preprocess
    x, m, new_annotation = preprocess.crop(img_arr, annotation=annotation)
    norm = preprocess.truncation_normalization(x, m)

    step1 = preprocess.median_denoise(norm, disk_radius=3)
    step2 = preprocess.unsharp_enhance(step1, radius=1.0, amount=1.5)
    step3 = preprocess.morphological_tophat(step2, selem_radius=15)
    step4 = preprocess.non_local_means_denoise(step3, patch_size=5, patch_distance=6, h_factor=0.8)
    step5 = preprocess.wavelet_enhancement(step4, wavelet='db8', level=1)
    final = preprocess.clahe(step5, clip_limit=0.02)
    disp = preprocess.normalize_for_display(final)
    disp = np.nan_to_num(disp)
    # disp = preprocess.draw_bbox_grayscale(disp.copy(), new_annotation, color=255, thickness=5)

    img_png_path_pre = os.path.join(save_dir, folder, f"{basename}_{idx}_preprocessed.png")
    if os.path.exists(img_png_path_pre): continue
    Image.fromarray(disp).save(img_png_path_pre)

    # ask
    history = [(
        f'Picture 1: <img>{img_png_path_pre}</img>\n这是什么?', 
        prompt.generate_mammogram_description(
            laterality=annotation['laterality'],
            view_position=annotation['view_position'],
            breast_density=annotation['breast_density'],
            breast_birads=annotation['breast_birads'],
            finding_categories=annotation['finding_categories'],
            finding_birads=annotation['finding_birads'],
            width=new_annotation['width'],
            height=new_annotation['height'],
            xmin=new_annotation['xmin'],
            ymin=new_annotation['ymin'],
            xmax=new_annotation['xmax'],
            ymax=new_annotation['ymax'],
        )
    )]

    query = tokenizer.from_list_format([
        {'image': img_png_path_pre},
        {'text': """请严格标注并框选图像中所有细小、圆形的可疑肿块（Mass）或可疑钙化灶（Suspicious Calcification）区域，输出对应的检测框坐标，以便后续诊断分析, 检测框应紧贴所检测到的目标。"""}
    ])

    response, history = model.chat(tokenizer, query=query, history=history)
    with open(log_path, "a", encoding="utf-8") as f:
        f.write(f"Response {idx}: {response}\n")
        f.write(f"History {idx}: {history}\n")
        f.write("\n")

    image = tokenizer.draw_bbox_on_latest_picture(response, history)
    if image:
        image.save(os.path.join(save_dir, folder, f"{basename}_{idx}_bbox.png"))
    else:
        print("No bbox")

    # Show
    try:
        folder = annotation['study_id']
        basename = annotation['image_id']
        img_png_path_pre = os.path.join(save_dir, folder, f"{basename}_{idx}_preprocessed.png")
        img_png_path_bbox = os.path.join(save_dir, folder, f"{basename}_{idx}_bbox.png")

        img1 = np.array(Image.open(img_png_path_pre))
        img1_bbox = preprocess.draw_bbox_grayscale(img1.copy(), new_annotation, color=255, thickness=5)

        img2 = np.array(Image.open(img_png_path_bbox))

        img3 = preprocess.draw_bbox_grayscale(img2.copy(), new_annotation, color=255, thickness=5)

        fig, axs = plt.subplots(1, 3, figsize=(9, 6))
        
        axs[0].imshow(img1_bbox, cmap='gray')
        axs[0].set_title("Old BBox")
        axs[0].axis("off")

        axs[1].imshow(img2, cmap='gray')
        axs[1].set_title("New Bbox")
        axs[1].axis("off")

        axs[2].imshow(img3, cmap='gray')
        axs[2].set_title("New BBox + Old BBox")
        axs[2].axis("off")

        plt.tight_layout()
        plt.show()

        print(f"{idx} Success at:", img_path)
    except: 
        print(f"{idx} Error at: ", img_path)

    break


# Experiment

In [ ]:
idx = 3
img_path, annotation = unique_tuples[idx]
folder = annotation['study_id']
os.makedirs(os.path.join(save_dir, folder), exist_ok=True)

basename = annotation['image_id']
img_png_path = os.path.join(save_dir, folder, f"{basename}.png")

ds = pydicom.dcmread(img_path)
img_arr = ds.pixel_array.astype(np.float32)

x, m, new_annotation = preprocess.crop(img_arr, annotation=annotation)
norm = preprocess.truncation_normalization(x, m)

step1 = preprocess.median_denoise(norm, disk_radius=3)
step2 = preprocess.unsharp_enhance(step1, radius=1.0, amount=1.5)
step3 = preprocess.morphological_tophat(step2, selem_radius=15)
step4 = preprocess.non_local_means_denoise(step3, patch_size=5, patch_distance=6, h_factor=0.8)
step5 = preprocess.wavelet_enhancement(step4, wavelet='db8', level=1)
final = preprocess.clahe(step5, clip_limit=0.02)
disp = preprocess.normalize_for_display(final)
disp = np.nan_to_num(disp)

img_png_path_pre = os.path.join(save_dir, folder, f"{basename}_{idx}_preprocessed.png")
Image.fromarray(disp).save(img_png_path_pre)

In [ ]:
query = tokenizer.from_list_format([
    {'image': img_png_path_pre},
    {'text': """请描述这张图像及其特征。"""}
])
response, history = model.chat(tokenizer, query=query, history=None)
print(response)

In [ ]:
image = tokenizer.draw_bbox_on_latest_picture(response, history)
if image:
    image.save(os.path.join(save_dir, folder, f"{basename}_{idx}_bbox.png"))
else:
    print("No bbox")

try:
    folder = annotation['study_id']
    basename = annotation['image_id']
    img_png_path_pre = os.path.join(save_dir, folder, f"{basename}_{idx}_preprocessed.png")
    img_png_path_bbox = os.path.join(save_dir, folder, f"{basename}_{idx}_bbox.png")

    img1 = np.array(Image.open(img_png_path_pre))
    img1_bbox = preprocess.draw_bbox_grayscale(img1.copy(), new_annotation, color=255, thickness=5)

    img2 = np.array(Image.open(img_png_path_bbox))

    img3 = preprocess.draw_bbox_grayscale(img2.copy(), new_annotation, color=255, thickness=5)

    fig, axs = plt.subplots(1, 3, figsize=(9, 6))
    
    axs[0].imshow(img1_bbox, cmap='gray')
    axs[0].set_title("Old BBox")
    axs[0].axis("off")

    axs[1].imshow(img2, cmap='gray')
    axs[1].set_title("New Bbox")
    axs[1].axis("off")

    axs[2].imshow(img3, cmap='gray')
    axs[2].set_title("New BBox + Old BBox")
    axs[2].axis("off")

    plt.tight_layout()
    plt.show()

    print(f"{idx} Success at:", img_path)
except: 
    print(f"{idx} Error at: ", img_path)